In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import MisrToolkit as mtk # needs python3.6
# from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob
import datetime as dt

In [2]:
# check python version
t1 = dt.datetime.now()
sys.version

'3.6.9 (default, Dec  8 2021, 21:08:43) \n[GCC 8.4.0]'

In [3]:
# load training dataset

home_dir = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data"
in_csv = "myMethod_merged_april_2016_9cam_4bands_training_dataset.csv"
# in_csv = "oldMethod_MisrAtmCalibratedModel_april_2016_training_dataset.csv"

ds_fp = os.path.join(home_dir, in_csv)
os.path.isfile(ds_fp)

True

In [4]:
# read data

in_ds = pd.read_csv(ds_fp)
in_ds.head(5)

,path,orbit,block,line,sample,lat,lon,Da_r,Ca_r,Ba_r,Aa_r,An_r,An_g,An_b,An_nir,Af_r,Bf_r,Cf_r,Df_r,mean_ATM_roughness
0,0,0,0,0.00,0.00,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00
1,152,86922,9,75.15,1205.60,74.422639,215.545163,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.97


In [5]:
# remove lines with zero
# Note: we must put each condition within parenthesis

# we use 3 filters to make sure we find every zero row and copy the filterd DF to new one

# in_ds = in_ds.loc[(in_ds['#path']!=0) &
#                  (in_ds[' orbit'] !=0) &
#                  (in_ds[' img_block'] !=0)]

in_ds = in_ds.loc[(in_ds['path']!=0) &
                 (in_ds['orbit'] !=0) &
                 (in_ds['block'] !=0)]

in_ds.head()

,path,orbit,block,line,sample,lat,lon,Da_r,Ca_r,Ba_r,Aa_r,An_r,An_g,An_b,An_nir,Af_r,Bf_r,Cf_r,Df_r,mean_ATM_roughness
1,152,86922,9,75.15,1205.60,74.422639,215.545163,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.19
2,152,86922,9,74.95,1205.52,74.422351,215.546735,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.72
3,152,86922,9,74.76,1205.45,74.422064,215.548307,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,6.33
4,152,86922,9,74.92,1205.51,74.422294,215.547049,1.03353,0.83645,0.71008,0.61530,0.58973,0.48393,0.76573,-99.0,0.69955,0.94627,1.39609,2.15582,5.97
5,152,86922,9,75.10,1205.71,74.422393,215.544545,1.03203,0.84397,0.71459,0.62433,0.59725,0.47642,0.76723,-99.0,0.70105,0.92672,1.39007,2.15582,5.18


In [6]:
in_ds.shape

(2924456, 20)

In [7]:
# subset necessary columns == 3 columns

ds_3cam = in_ds[["path", "orbit", "block", "line", "sample", "lat", "lon", "Ca_r", "Cf_r", "An_r", "mean_ATM_roughness"]] # add path & block here

# ds_3cam = in_ds[["#path", " orbit", " img_block", " line", " sample", " lat", " lon", " an", " ca", " cf", " rms"]] # add path & block here

ds_3cam.columns

Index(['path', 'orbit', 'block', 'line', 'sample', 'lat', 'lon', 'Ca_r',
       'Cf_r', 'An_r', 'mean_ATM_roughness'],
      dtype='object')

In [8]:
# rename last column

ds_3cam.rename(columns={"mean_ATM_roughness":"rms"}, inplace=True)
ds_3cam.columns

/home/ehsan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['path', 'orbit', 'block', 'line', 'sample', 'lat', 'lon', 'Ca_r',
       'Cf_r', 'An_r', 'rms'],
      dtype='object')

In [9]:
# compute date-time from orbit

# orbit_list = ds_3cam[' orbit'].to_list()
orbit_list = ds_3cam['orbit'].to_list()

print(type(orbit_list))

# for x in orbit_list:
#     if x<1:
#         orbit_list.remove(x)

orbit_list_noZero = [i for i in orbit_list if i>0]

# print(len(orbit_list_noZero))
# print(min(orbit_list_noZero))
# print(max(orbit_list_noZero))

<class 'list'>


In [10]:
# use MTK and change orbit to date-time

orbit_start_stop_list = []

for orbiti in orbit_list_noZero:
    orbit_start_stop = mtk.orbit_to_time_range(orbiti)
    orbit_start_stop_list.append(orbit_start_stop)

In [11]:
# change list to np.array and extract start and stop columns 

orbit_start = np.array(orbit_start_stop_list)[:,0]
orbit_stop = np.array(orbit_start_stop_list)[:,1]

In [12]:
# add a date-time column to DF

ds_3cam['orbit_start'] = orbit_start
ds_3cam['orbit_stop'] = orbit_stop
ds_3cam.head(5)

/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop
1,152,86922,9,75.15,1205.60,74.422639,215.545163,0.84397,1.39007,0.59725,5.19,2016-04-21T05:32:36Z,2016-04-21T07:11:29Z
2,152,86922,9,74.95,1205.52,74.422351,215.546735,0.83645,1.39609,0.58973,5.72,2016-04-21T05:32:36Z,2016-04-21T07:11:29Z
3,152,86922,9,74.76,1205.45,74.422064,215.548307,0.83645,1.39609,0.58973,6.33,2016-04-21T05:32:36Z,2016-04-21T07:11:29Z
4,152,86922,9,74.92,1205.51,74.422294,215.547049,0.83645,1.39609,0.58973,5.97,2016-04-21T05:32:36Z,2016-04-21T07:11:29Z
5,152,86922,9,75.10,1205.71,74.422393,215.544545,0.84397,1.39007,0.59725,5.18,2016-04-21T05:32:36Z,2016-04-21T07:11:29Z


In [13]:
# change string to datetime format and apply it on specific DF column

ds_3cam['orbit_start'] = pd.to_datetime(ds_3cam['orbit_start'])
ds_3cam['orbit_stop'] = pd.to_datetime(ds_3cam['orbit_stop'])
ds_3cam.head()

/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,path,orbit,block,line,sample,lat,lon,Ca_r,Cf_r,An_r,rms,orbit_start,orbit_stop
1,152,86922,9,75.15,1205.60,74.422639,215.545163,0.84397,1.39007,0.59725,5.19,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
2,152,86922,9,74.95,1205.52,74.422351,215.546735,0.83645,1.39609,0.58973,5.72,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
3,152,86922,9,74.76,1205.45,74.422064,215.548307,0.83645,1.39609,0.58973,6.33,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
4,152,86922,9,74.92,1205.51,74.422294,215.547049,0.83645,1.39609,0.58973,5.97,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00
5,152,86922,9,75.10,1205.71,74.422393,215.544545,0.84397,1.39007,0.59725,5.18,2016-04-21 05:32:36+00:00,2016-04-21 07:11:29+00:00


In [14]:
ds_3cam.dtypes

path                         int64
orbit                        int64
block                        int64
line                       float64
sample                     float64
lat                        float64
lon                        float64
Ca_r                       float64
Cf_r                       float64
An_r                       float64
rms                        float64
orbit_start    datetime64[ns, UTC]
orbit_stop     datetime64[ns, UTC]
dtype: object

In [17]:
# write out dataset with date-time

output_filename = in_csv.split('.')[0]+"_datetime.csv"

output_file_fp = os.path.join(home_dir, output_filename)
ds_3cam.to_csv(output_file_fp, encoding='utf-16')
output_file_fp

'/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data/myMethod_merged_april_2016_9cam_4bands_training_dataset_datetime.csv'

In [18]:
t2 = dt.datetime.now()
runtime = t2-t1
runtime

datetime.timedelta(0, 324, 173710)